In [1]:
import tensorflow as tf
physicalDevices = tf.config.list_physical_devices('GPU') 
if physicalDevices:
    tf.config.experimental.set_memory_growth(physicalDevices[0], True)  
from tensorflow import keras
from tensorflow.keras import layers
from data_preprocessor import DataGenerator
from gestures import GESTURES


import datetime

In [2]:
batch_size=64
epochs=50
input_shape = (20,21,3)
classes = len(GESTURES)
trainGen = DataGenerator('gesture_dataset.h5')

In [3]:
inputs = layers.Input(shape=input_shape)
x = inputs
x = layers.TimeDistributed(layers.Flatten())(x)
x = layers.TimeDistributed(layers.Dropout(0.5))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dropout(0.5))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dropout(0.3))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dropout(0.2))(x)
x = layers.GRU(64)(x)
output = layers.Dense(classes, activation='softmax',name='Output')(x)
model = keras.models.Model(inputs, output)
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20, 21, 3)]       0         
_________________________________________________________________
time_distributed (TimeDistri (None, 20, 63)            0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 20, 63)            0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 20, 64)            4096      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 20, 64)            0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 20, 64)            4160      
_________________________________________________________________
time_distributed_5 (TimeDist (None, 20, 64)            0     

In [4]:
model.fit(trainGen,epochs=epochs)

Epoch 1/50
90/90 [==============================] - 18s 6ms/step - loss: 1.7400 - accuracy: 0.3228
Epoch 2/50
90/90 [==============================] - 1s 6ms/step - loss: 0.3795 - accuracy: 0.8837
Epoch 3/50
90/90 [==============================] - 1s 7ms/step - loss: 0.1064 - accuracy: 0.9681
Epoch 4/50
90/90 [==============================] - 1s 6ms/step - loss: 0.0770 - accuracy: 0.9826
Epoch 5/50
90/90 [==============================] - 1s 6ms/step - loss: 0.0433 - accuracy: 0.9885
Epoch 6/50
90/90 [==============================] - 1s 6ms/step - loss: 0.0390 - accuracy: 0.9906
Epoch 7/50
90/90 [==============================] - 1s 6ms/step - loss: 0.0355 - accuracy: 0.9944
Epoch 8/50
90/90 [==============================] - 1s 6ms/step - loss: 0.0211 - accuracy: 0.9966
Epoch 9/50
90/90 [==============================] - 1s 6ms/step - loss: 0.0197 - accuracy: 0.9974
Epoch 10/50
90/90 [==============================] - 1s 6ms/step - loss: 0.0349 - accuracy: 0.9944
Epoch 11/50
90/90 

In [5]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tfliteModel = converter.convert()
modelPath = "saved_models/MODEL-{}.tflite".format(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
with open(modelPath, 'wb') as f:
  f.write(tfliteModel)

INFO:tensorflow:Assets written to: C:\Users\17ami\AppData\Local\Temp\tmptou5mtq5\assets
INFO:tensorflow:Assets written to: C:\Users\17ami\AppData\Local\Temp\tmptou5mtq5\assets
